In [0]:
!git clone https://github.com/tyiannak/pyAudioAnalysis.git

In [0]:
cd pyAudioAnalysis/

In [0]:
pip install -r ./requirements.txt

In [0]:
cd pyAudioAnalysis/

In [0]:
pip install -e .

In [0]:
!sudo apt-get install ffmpeg
!sudo apt-get install python3-magic
!pip install python-magic

In [0]:
# extraction of features using pyAudioAnalysis library
def feature_extraction(filename,txtfile):
  features = []; mt_d = []; st_features = []; cycle_idx = [0]; i = 0; class_label = []; exception = False; f_names = [];
  failure = '';

  f = open(txtfile, "r")
  person = AudioSegment.from_wav(filename)
  for x in f:
    x = x.split();
    t1 = float(x[0]) * 1000; t2 = float(x[1]) * 1000;   #obtaining the cycles from the audio files
    s = person[t1:t2];
    s.export('/content/drive/My Drive/respiratory-sound-database/newcycle.wav', format="wav"); #Exports to a wav file in the current path
    try:
      [w, Fs] = data, sr = librosa.load("/content/drive/My Drive/respiratory-sound-database/newcycle.wav", sr=None)
      F, sF, f_names = MidTermFeatures.mid_feature_extraction(w, Fs, (t2-t1)*Fs, (t2-t1)*Fs, 0.050*Fs, 0.025*Fs)  #extracting the features
      n = int(x[2])*2 + int(x[3])*1;
      class_label.append(n);
      features.append(np.transpose(F));
      st_features.append(np.transpose(sF))
      cycle_idx.append(sF.shape[1]+cycle_idx[i]);
      mt_d.append(F.shape[1]);
      i = i+1;
    except Exception as e:
        print("Error encountered while parsing this cycle in ", filename)
        exception = True;

  if exception == True:
    if len(features) < 1:
      failure = filename;
      return features, f_names, st_features, cycle_idx, class_label, failure
    if len(features) < 2:
      return features, f_names, st_features, cycle_idx, class_label, failure

  features = np.concatenate(features, axis=0)
  st_features = np.concatenate(st_features, axis=0)
  
  return features, f_names, st_features, cycle_idx, class_label, failure

In [0]:
mypath = "/content/drive/My Drive/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/"
filenames = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.wav'))]
filenames[0].split('.')[0]+'.txt'

In [0]:
from datetime import datetime
from os import listdir
from os.path import isfile, join

# Feature extraction
import numpy as np
import librosa
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures,MidTermFeatures
from pydub import AudioSegment
import matplotlib.pyplot as plt

mypath = "/content/drive/My Drive/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/"
filenames = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.wav'))] 
filepaths = [join(mypath, f) for f in filenames] # full paths of files


extracted_features = []; st_features = []; cycle_idx = []; class_label = []; fail = [];

# Iterate through each sound file and extract the features
for file_name in filepaths:
  print(file_name)
  f, f_names, sF, idx, c_label, failure = feature_extraction(file_name,file_name.split('.')[0]+'.txt');
  extracted_features.append(f)
  st_features.append(sF)
  cycle_idx.append(idx)
  class_label.append(c_label)
  fail.append(failure)

In [0]:
n_f = []; l = len(extracted_features)
for i in range(l):
  if extracted_features[i] != []:
    n_f.append(extracted_features[i])
print(len(n_f))
n_f = np.concatenate(n_f, axis=0)

In [0]:
class_lab = np.concatenate(class_label,axis=0)

In [0]:
f = []; length = len(fail)
for i in range(length):
  if fail[i] != '':
    f.append(fail[i])

In [0]:
l = len(extracted_features); names = []; p_no = [];
for i in range(l):
  if extracted_features[i] != []:
    for j in range(extracted_features[i].shape[0]):
      names.append(filenames[i])
      p_no.append(filenames[i].split('_')[0])

In [0]:
import pandas as pd
import pickle

data_dict = {}
pickle_out = open("/content/drive/My Drive/respiratory-sound-database/data.pickle","wb")
df = pd.DataFrame(n_f, columns=f_names)
df['Patient No'] = p_no; df['Name'] = names; df['Class'] = class_lab;
df.to_csv ('/content/drive/My Drive/respiratory-sound-database/Respiratory_Data.csv', index = False, header=True)
data_dict['csv_data'] = df
data_dict['short_term_features'] = st_features
data_dict['cycle_index'] = cycle_idx
data_dict['Failures'] = f
pickle.dump(data_dict,pickle_out)
pickle_out.close()

Classifiers of respiratory data

In [0]:
import sklearn
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE

In [0]:
import pandas as pd
import pickle

pickle_in = open("/content/drive/My Drive/respiratory-sound-database/data.pickle","rb")
data = pickle.load(pickle_in)
dfr = pd.read_csv('/content/drive/My Drive/respiratory-sound-database/Respiratory_Data.csv')
# print(data.keys())
pickle_in.close()

In [0]:
X = dfr.to_numpy()
labels = dfr['Class'].to_numpy()

In [0]:
one=0; two=0;three=0;four=0;
for i in range(labels.shape[0]):
  if labels[i] == 0:
    one = one + 1
  elif labels[i] == 1:
    two = two + 1
  elif labels[i] == 2:
    three = three + 1
  else:
    four = four + 1

le = LabelEncoder()
i_labels = le.fit_transform(labels)

In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,labels,test_size=0.3, random_state=42)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [0]:
# Running SVM classifier
Cs = [2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10)]
gamma = [2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]


param_grid = {'C': Cs,  
                'gamma': gamma, 
                'kernel': ['rbf'],
                'decision_function_shape':['ovr'],
                'class_weight': ['balanced']}  
grid1 = GridSearchCV(SVC(), param_grid,cv=3,n_jobs=-1, verbose = 3) 

# fitting the model for grid search 
grid1.fit(x_train, y_train)

# print best parameter after tuning 
print(grid1.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid1.best_estimator_)

grid_predictions = grid1.predict(x_test) 

# print classification report 
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test,grid_predictions))

print(sklearn.metrics.confusion_matrix(y_test,grid_predictions))

In [0]:
# Running XgBoost
import xgboost as xgb

print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=3,num_class=2, n_estimators=15, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)
print(accuracy_score(y_test,xgb_pred))

In [0]:
#Running ANN
# One hot the labels
le = LabelEncoder()
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)



# Custom model for classification
model = Sequential()

model.add(Dense(24, activation='relu', input_shape=(24,),kernel_initializer='random_normal'))

model.add(Dense(12, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(300, activation='relu'))

#model.add(Dense(150, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(75, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(64, activation='relu',kernel_initializer='random_normal')) 
#model.add(Dropout(0.2))


#model.add(Dense(16, activation='relu') )

model.add(Dense(2, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

model.summary()



# Load the trained model if required
#model_file = '/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/LogMel_MFCC_Chroma_CNN/Try6_Unlinked/model_40_230.h5'
#model.load_weights(model_file)

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

In [0]:
#train model
num_epochs = 1000
num_batch_size = 128



callbacks = [
    ModelCheckpoint(
        filepath='/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/Chroma_ANN_Binary/Chroma_ANN_Binary_{epoch:02d}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), callbacks=callbacks, verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)